<h1>Math 265 Linear Algebra Homework 4</h1>

<h2>Matrix multiplication</h2>

In [1]:
def MatrixDim(A):
    #
    # This function is supposed to return the dimensions of matrix A
    #
    M265rows = len(A)
    if (M265rows <= 0):
        return[-1,-1]
    M265cols = len(A[0])
    # we will check that this is indeed matrix i.e. number of cols in every row is the same
    for i in range(M265rows):
        if (len(A[i]) != M265cols):
            return [-1,-1]
    return [M265rows, M265cols]  

def nPrint(A):
    for i in range(len(A)):
        print(A[i])
    return True

def MatrixMult(A,B):
    #
    # This function should multiply the matrices A and B
    # start by creating an empty array
    [rowsA,colsA] = MatrixDim(A)
    [rowsB,colsB] = MatrixDim(B)
    if ((colsA != rowsB) and (colsA > 0)):
        # if matrix multiplication cannot be done return an empty array
        return []
        
    M265Result = [[0 for k in range(colsB)] for r in range(rowsA)]
    for rw in range(rowsA):
        for cl in range(colsB):
            tmp =0
            for k in range(colsA):
                tmp = tmp + A[rw][k]*B[k][cl]
            M265Result[rw][cl]=tmp
    return M265Result

<h2>Back Substitution Programming<h2>

In [2]:
def M265isEchelon(A):
    #
    # given a matrix A this function must check if the matrix is in Echelon form
    # the function should return True if the matrix A is in Echelon form 
    # else the function should return false
    #
    M265_isEcheclon = False
    [rowsA,colsA]=MatrixDim(A)
    leading = [colsA+1+rw for rw in range(rowsA)]
    for rw in range(rowsA):
        for cl in range(colsA):
            if (A[rw][cl] != 0):
                leading[rw] = cl
                break
    M265_isEchelon = True
    for rw in range(rowsA-1):
        if (leading[rw] >= leading[rw+1]):
            M265_isEchelon = False
    return M265_isEchelon

def M265isREch(A):
    #
    # given a matrix A this function returns an array with one negative
    # value depending on which check failed if A is not in reduced Echelon form
    # otherwise the result is an array of col positions of leading variables
    [rowsA,colsA] = MatrixDim(A)
    leading = [colsA+1+rw for rw in range(rowsA)]
    posLeading = []
    # first we find leading coefficients and check that those equal 1
    for rw in range(rowsA):
        for cl in range(colsA):
            if (A[rw][cl] != 0):
                leading[rw] = cl
                if (A[rw][cl]!=1):
                    return [-1]
                posLeading.append(cl)
                break
    # second we check every leading variable is to the right of 
    # the leading variable in the above row
    M265_isEchelon = True
    for rw in range(rowsA-1):
        if (leading[rw] >= leading[rw+1]):
            M265_isEchelon = False
    if (not M265_isEchelon):
        return [-2]
    # lastly we check that for a given leading variable the coefficients in any
    # other equation for that variable is zero
    for i in range(len(posLeading)):
        for rw in range(rowsA):
            if ((rw != i) and (A[rw][posLeading[i]]!=0)):
                return [-3]
    return posLeading
    
    
def M265BackSubs(A,b):
    #
    # in this function you are supposed to solve a system of linear equations
    # with matrix A and constant b
    # the result should be an array of vectors M265_solSet
    # the first entry of M265_solSet is a particular solution
    # the remaining entries are vectors describing the homogeneous solution
    #
    ##################
    #
    # First let us create a vector for the particular solution
    #
    [rowsA,colsA] = MatrixDim(A)
    if ((rowsA < 0) or (colsA<0)):
        return []
    if (len(b)!=rowsA):
        return []
    LPos = M265isREch(A)
    #if (len(LPos)<=0):
    #    return [-1]
    M265_partSol = [0 for i in range(colsA)]
    for i in range(len(LPos)):
        M265_partSol[LPos[i]] = b[i]
    if (len(LPos)<len(b)):
        for i in range(len(LPos),len(b)):
            if (b[i]!=0):
                M265_partSol=[]
        
    M265_homgSol = [0 for i in range(len(A[0]))]
    
    Free=[0 for i in range(colsA - len(LPos))]
    freePos =0
    basicPos =0
    for i in range(colsA):
        if ((basicPos >= len(LPos)) or (i<LPos[basicPos]))  :
            Free[freePos] = i
            freePos+=1
        else:
            basicPos+=1

    M265_homgSol = [[0 for c in range(colsA)] for i in range(len(Free))]

    for c in range(len(Free)):
        M265_homgSol[c][Free[c]]=1
        for l in range(len(LPos)):
            M265_homgSol[c][LPos[l]]=-A[l][Free[c]]
            
    return [M265_partSol,M265_homgSol]

<h2>Gaussian Elimination</h2>

In [3]:
def M265Swap(rowi,rowj,order):
    if ((rowi >= order) or (rowj>=order)):
        return []
    res = [[0 for i in range(order)] for j in range(order)]
    for i in range(order):
        res[i][i]=1
    res[rowi][rowj]=1
    res[rowj][rowi]=1
    res[rowi][rowi]=0
    res[rowj][rowj]=0
    return res

def M265cM(row,const,order):
    if (row >= order):
        return []
    res = [[0 for i in range(order)] for j in range(order)]
    for i in range(order):
        res[i][i]=1
    res[row][row]=const
    return res

def M265LC(rowi,rowj,const,order):
    # set rowi as rowi+const*rowj
    if ((rowi >= order) or (rowj>=order) or (rowi==rowj)):
        return []
    res = [[0 for i in range(order)] for j in range(order)]
    for i in range(order):
        res[i][i]=1
    res[rowi][rowj]=const
    return res

def M265GaussianElimination(A,debug=False):
    resEliminations = []
    [rowsA,colsA]=MatrixDim(A)
    order=rowsA
    EP = [[0 for i in range(rowsA)] for j in range(rowsA)]
    if (debug):
        nPrint(A)
    rw = 0
    cl = 0
    for c in range(colsA):
        hasPivot = False
        # first we check if the column under question has a pivot
        for r in range(rw,rowsA):
            if (A[r][cl]!=0):
                hasPivot = True
                if (r!=rw):
                    EP =M265Swap(rw,r,order)
                    A = MatrixMult(EP,A)
                    resEliminations.append(EP)
                    if (debug):
                        print("\n")
                        print("Swap row ",rw,"with row", r)
                        nPrint(A)
                break # if there is a pivot 
        if (hasPivot):
            # if there is a pivot first scale it to one if the pivot is not one
            if (A[rw][cl]!=1):
                EP = M265cM(rw,1.0/A[rw][cl],order)       
                resEliminations.append(EP)
                A = MatrixMult(EP,A)
                if (debug):
                    print("\n")
                    print("Scale row", rw)
                    nPrint(A)

            # make the respective entries in the column zero except for the row that contains the pivot (r==rw)
            # further if a row has already entry zero at that column skip that row as the resulting scaling
            # is simply the identity matrix (A[r][cl]==0)
            for r in range(rowsA):
                if ((r==rw) or (A[r][cl]==0)):
                    continue
                EP = M265LC(r,rw,-1.0*A[r][cl],order)
                resEliminations.append(EP)
                A = MatrixMult(EP,A)
                if(debug):
                    print("\n")
                    print("Add multiple of row",rw,"to row",r )
                    nPrint(A)
            rw += 1 # if there was a pivot in this column for the remaining rows up we search for pivots
            # only on rows that are below it
        cl += 1
    return resEliminations

<h1> Homework 4 </h1>

The notebook send earlier today describes a vector space V and operations on that vector space. 

<b>Task 1:</b> Implement those (you may disregard the homework from last week that also required you to write operations of a vector space). 

The notebook contains four b1, b2, b3 and b4 vectors from V. They are linearly dependent. You must set up a system of linear equations whose set of solutions is going to tell you that the vectors are linearly dependent

<b>Task 2:</b> write the matrix and vector of constants that when solved identifies that vectors b1, b2, b3 and b4 are linearly dependent.

Starting with b1,b2,b3 and b4 by removing one vector from those you can show that the vectors are linearly independent. This is again done with a system of linear equations and analyzing its set of solutions.

<b>Task 3:</b> as in Task2 write the system of linear equations and a vector of constants that when solved will tell you that the vectors you chose are linearly independent.

In your first homework you were given matrices. From the first of those matrices, take the first three values in row zero and treat them as a vector in V. Call this vector u.

<b>Task 4:</b> Set up a system of linear equation (matrix and vector of constants) that when solved will give the coordinates of u in the base from Task 3.

<b>Task 5:</b> Given any basis B of V and vector w in V, write a function that returns the coordinates of w in B.

--------------------------------------------------------------------------------------------------------------------------

<h3>The Vector Space: V</h3>

$ u = \left(\begin{array}{c} x\\y\\z\end{array}\right)$ $ v = \left(\begin{array}{c} s\\t\\r\end{array}\right)$

$$u\oplus v 
 = \left(\begin{array}{c} x\\y\\z\end{array}\right)
     \oplus \left(\begin{array}{c} s\\t\\r\end{array}\right)
 = \left(\begin{array}{c} x+s-2\\y+t\\z+r+4\end{array}\right)$$

$$\alpha \odot \vec{u}
    =\alpha \odot\left(\begin{array}{c} x\\y\\z\end{array}\right)
    =\alpha \odot\left(\begin{array}{c} 
            \alpha x-2\alpha +2
            \\\alpha y
            \\\alpha z +4\alpha - 4\end{array}\right)$$

In [69]:
b1 =[1,3,4]
b2 = [0,0,0]
b3 = [2,0,-4]
b4 = [-2,0,4]

<h3> Task 1: Vector Space Implementation </h3>

In [70]:
def VS_sum(m1,m2):
    return [m1[0] + m2[0] - 2, m1[1] + m2[1], m1[2] + m2[2] + 4]

def VS_mult(a,m):
    return [(a*m[0])-(2*a)+2, a*m[1], (a*m[2])-(4*a)-4]

def VS_zero(m):
    return VS_mult(0,m)

def VS_additiveInverse(m):
    return VS_mult(-1,m)

<h3> Task 2: Showing Linear Dependency </h3>

In [71]:
zv = VS_zero(b1)
AM = [[1-zv[0],0-zv[0],2-zv[0],-2-zv[0]],[3-zv[1],0-zv[1],0-zv[1],0-zv[1]],[4-zv[2],0-zv[2],-4-zv[2],4-zv[2]]]
print("Coefficent Matrix: ")
nPrint(AM)
b = [0,0,0]

Coefficent Matrix: 
[-1, -2, 0, -4]
[3, 0, 0, 0]
[8, 4, 0, 8]


$$\begin{bmatrix} -1 & -2 & 0 & -4 & | & 0 \\ 3 & 0 & 0 & 0 & | & 0 \\ 8 & 4 & 0 & 8 & | & 0 \end{bmatrix}$$

In [72]:
for elementary in M265GaussianElimination(AM):
    AM = MatrixMult(elementary, AM)

In [73]:
nPrint(AM)

[1.0, 0.0, 0.0, 0.0]
[0.0, 1.0, 0.0, 2.0]
[0.0, 0.0, 0.0, 0.0]


True

In [74]:
M265BackSubs(AM,b)

[[0, 0, 0, 0], [[-0.0, -0.0, 1, 0], [-0.0, -2.0, 0, 1]]]

<em>conclude</em>

Removing <b>b1</b>.

In [75]:
AM = [0-zv[0],2-zv[0],-2-zv[0]],[0-zv[1],0-zv[1],0-zv[1]],[0-zv[2],-4-zv[2],4-zv[2]]
nPrint(AM)
for elementary in M265GaussianElimination(AM):
    AM = MatrixMult(elementary, AM)
nPrint(AM)
M265BackSubs(AM,b)

[-2, 0, -4]
[0, 0, 0]
[4, 0, 8]
[1.0, 0.0, 2.0]
[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0]


[[0, 0, 0], [[-0.0, 1, 0], [-2.0, 0, 1]]]

Removing <b>b2</b>.

In [76]:
AM = [[1-zv[0],2-zv[0],-2-zv[0]],[3-zv[1],0-zv[1],0-zv[1]],[4-zv[2],-4-zv[2],4-zv[2]]]
nPrint(AM)
for elementary in M265GaussianElimination(AM):
    AM = MatrixMult(elementary, AM)
nPrint(AM)
M265BackSubs(AM,b)

[-1, 0, -4]
[3, 0, 0]
[8, 0, 8]
[1.0, 0.0, 0.0]
[0.0, 0.0, 1.0]
[0.0, 0.0, 0.0]


[[0, 0, 0], [[-0.0, 1, -0.0]]]

Removing <b>b3</b>.

In [77]:
AM = [[1-zv[0],0-zv[0],-2-zv[0]],[3-zv[1],0-zv[1],0-zv[1]],[4-zv[2],0-zv[2],4-zv[2]]]
nPrint(AM)
for elementary in M265GaussianElimination(AM):
    AM = MatrixMult(elementary, AM)
nPrint(AM)
M265BackSubs(AM,b)

[-1, -2, -4]
[3, 0, 0]
[8, 4, 8]
[1.0, 0.0, 0.0]
[0.0, 1.0, 2.0]
[0.0, 0.0, 0.0]


[[0, 0, 0], [[-0.0, -2.0, 1]]]

Removing <b>b4</b>.

In [79]:
AM = [[1-zv[0],0-zv[0],2-zv[0]],[3-zv[1],0-zv[1],0-zv[1]],[4-zv[2],0-zv[2],-4-zv[2]]]
nPrint(AM)
for elementary in M265GaussianElimination(AM):
    AM = MatrixMult(elementary, AM)
nPrint(AM)
M265BackSubs(AM,b)

[-1, -2, 0]
[3, 0, 0]
[8, 4, 0]
[1.0, 0.0, 0.0]
[0.0, 1.0, 0.0]
[0.0, 0.0, 0.0]


[[0, 0, 0], [[-0.0, -0.0, 1]]]